<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using OpenAI and LangChain's SQLDatabaseToolkit with SingleStore</h1>
    </div>
</div>

In [1]:
!pip cache purge --quiet

In [2]:
!pip install langchain langchain-community langchain-openai openai sqlalchemy-singlestoredb --quiet

In [3]:
import openai
import os

from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain_openai.chat_models import ChatOpenAI

In [4]:
model = "gpt-4o-mini"

openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(
    model = model,
    temperature = 0,
    verbose = False
)

In [6]:
connection_url = "singlestoredb://" + os.getenv("SINGLESTOREDB_URL")

In [7]:
db = SQLDatabase.from_uri(
    connection_url,
    include_tables = ["tick"]
)

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

agent_executor = create_sql_agent(
    llm = llm,
    toolkit = toolkit,
    max_iterations = 15,
    max_execution_time = 120,
    top_k = 1,
    verbose = False
)

In [8]:
error_string = "Could not parse LLM output:"

def run_agent_query(query, agent_executor, error_string):
    try:
        result = agent_executor.invoke(query, return_only_outputs = True)["output"]
    except Exception as e:
        error_message = str(e)
        # Check if the error message contains the specific string
        if error_string in error_message:
            # Extract the part after the specific string and strip backticks
            result = error_message.split(error_string)[1].strip().strip('`')
        else:
            result = f"Error occurred: {error_message}"
    return result

In [9]:
# First query
query1 = (
    "For each stock symbol, calculate the volatility as the difference\n"
    "between the highest recorded price and the lowest recorded price over time.\n"
    "Which stock symbol has the least volatility?"
)
result1 = run_agent_query(query1, agent_executor, error_string)
print(result1)

The stock symbol with the least volatility is FTR, with a volatility of 0.55.


In [10]:
# Second query with user input
query2 = input("Please enter your question:")
if not query2.strip():
    query2 = "How many rows are in the tick table?"
    print(f"No input provided. Using default query: '{query2}'")

result2 = run_agent_query(query2, agent_executor, error_string)
print(result2)

Please enter your question: 


No input provided. Using default query: 'How many rows are in the tick table?'
There are 22,367,162 rows in the tick table.


In [11]:
db = SQLDatabase.from_uri(
    connection_url,
    include_tables = ["tick", "portfolio", "stock_sentiment"]
)

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

agent_executor = create_sql_agent(
    llm = llm,
    toolkit = toolkit,
    max_iterations = 15,
    max_execution_time = 120,
    top_k = 1,
    verbose = False
)

In [12]:
# Third query
query3 = (
    "Taking all the stock symbols from the portfolio table,\n"
    "and using the latest value for each stock symbol from the tick table,\n"
    "calculate the grand total value of all the shares listed in the portfolio table."
)
result3 = run_agent_query(query3, agent_executor, error_string)
print(result3)

$44,540.60


In [13]:
# Fourth query
query4 = (
    "Using the symbol AAPL, show me the date and the most\n"
    "positive sentiment in the stock sentiment table and the\n"
    "current best price for this symbol from the tick table."
)
result4 = run_agent_query(query4, agent_executor, error_string)
print(result4)

On 2020-05-28, the most positive sentiment for AAPL was 0.331509, and the current best price for AAPL is 116.27.
